# Graph Oluşturma

Bu script donanımsal kısıtlardan dolayı colab de çalıştırılmıştır.

In [ ]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
from geopy.distance import great_circle
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# --- Adım 1: Tüm Yenilikçi Özellikleri Oluşturma ---

df = pd.read_csv('atlanta_cleaned_data.csv')
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df.dropna(subset=['Start_Time'], inplace=True)


# --- 1a: Şiddet Ağırlıklı Sokak Ağı Özelliği ---
print("--- Bölüm A: Şiddet Ağırlıklı Sokak Ağı Özelliği oluşturuluyor ---")
junction_df = df[df['Junction'] == True].dropna(subset=['Description', 'Street']).copy()
edge_list_with_severity = []
pattern = re.compile(r' on (.+?)(?: at | & )(.+?)(?: - | \.|,|$)')

for index, row in tqdm(junction_df.iterrows(), total=junction_df.shape[0]):
    match = pattern.search(row['Description'])
    if match:
        street1, street2 = match.group(1).strip(), match.group(2).strip()
        if len(street1) > 2 and len(street2) > 2:
            edge_list_with_severity.append(tuple(sorted((street1, street2))) + (row['Severity'],))

severity_edge_df = pd.DataFrame(edge_list_with_severity, columns=['Sokak_1', 'Sokak_2', 'Severity'])
severity_weighted_edges = severity_edge_df.groupby(['Sokak_1', 'Sokak_2']).agg(severity_sum=('Severity', 'sum')).reset_index()
G_severity_weighted = nx.from_pandas_edgelist(severity_weighted_edges, 'Sokak_1', 'Sokak_2', edge_attr='severity_sum')
degree_centrality_severity = {node: val for node, val in G_severity_weighted.degree(weight='severity_sum')}
df['Severity_Degree_Score'] = df['Street'].map(degree_centrality_severity).fillna(0)
print("-> 'Severity_Degree_Score' özelliği oluşturuldu.\n")


# --- 1b: Kaza Yayılım Ağı Özellikleri ---
print("--- Bölüm B: Kaza Yayılım Ağı Özellikleri oluşturuluyor ---")
df.sort_values(by='Start_Time', inplace=True)
df.reset_index(drop=True, inplace=True)

coords = df[['Start_Lat', 'Start_Lng']].values
times = pd.to_datetime(df['Start_Time']).values
distances_mi = df['Distance(mi)'].values
ids = df['ID'].values
MILE_TO_KM = 1.60934

propagation_edges = []
time_window = pd.Timedelta(hours=1)
for i in tqdm(range(1, len(df))):
    for j in range(i - 1, -1, -1):
        if (times[i] - times[j]) > time_window: break
        dist_km = great_circle(coords[i], coords[j]).km
        if dist_km <= (distances_mi[j] * MILE_TO_KM):
            propagation_edges.append((ids[j], ids[i]))

G_propagation = nx.DiGraph()
G_propagation.add_edges_from(propagation_edges)
df['In_Degree'] = df['ID'].map(G_propagation.in_degree).fillna(0)
df['Out_Degree'] = df['ID'].map(G_propagation.out_degree).fillna(0)
print("-> 'In_Degree' ve 'Out_Degree' özellikleri oluşturuldu.\n")

In [ ]:
# --- Adım 2: Nihai Modelleme ---
print("--- Nihai Modelleme Aşaması ---")


cols_to_drop = [
    'ID', 'Source', 'Description', 'Street', 'City', 'County', 'State', 'Zipcode',
    'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Start_Time',
    'End_Time', 'Start_Lat', 'Start_Lng', 'Weather_Condition', 'Wind_Direction',
    'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Distance(mi)'
]
df_model = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

X = df_model.drop('Severity', axis=1)
y = df_model['Severity']
X = pd.get_dummies(X, drop_first=True)

print(f"Nihai model için özellik sayısı: {X.shape[1]}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Veri {len(X_train)} eğitim, {len(X_test)} test satırına ayrıldı.")

# RandomForest

In [ ]:
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
print("Nihai modelin eğitimi tamamlandı.")

# --- Performans Değerlendirmesi ---
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"\nNihai Modelin Doğruluğu: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=['Ciddi Değil', 'Ciddi']))

In [ ]:
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)

# En önemli 20 özelliği al ve sırala
top_20_features = feature_importances.nlargest(20)

# Görselleştirme
plt.figure(figsize=(12, 10))
sns.barplot(x=top_20_features, y=top_20_features.index)
plt.title(f'Model için En Önemli 20 Özellik (Feature Importance)', fontsize=16)
plt.xlabel('Önem Skoru', fontsize=12)
plt.ylabel('Özellikler', fontsize=12)
plt.show()

# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

xgb_model = XGBClassifier(random_state=42, n_jobs=-1)
# Convert pandas DataFrames to NumPy arrays
xgb_model.fit(X_train.values, y_train.values)
print("Nihai modelin eğitimi tamamlandı.")

# --- Performans Değerlendirmesi ---
y_pred = xgb_model.predict(X_test.values)
accuracy = accuracy_score(y_test.values, y_pred)
precision = precision_score(y_test.values, y_pred)
recall = recall_score(y_test.values, y_pred)
f1 = f1_score(y_test.values, y_pred)

print(f"\nNihai Modelin Doğruluğu: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Sınıflandırma Raporu:")
print(classification_report(y_test.values, y_pred, target_names=['Ciddi Değil', 'Ciddi']))

In [ ]:
feature_importances = pd.Series(xgb_model.feature_importances_, index=X.columns)

# En önemli 20 özelliği al ve sırala
top_20_features = feature_importances.nlargest(20)

# Görselleştirme
plt.figure(figsize=(12, 10))
sns.barplot(x=top_20_features, y=top_20_features.index)
plt.title(f'Model için En Önemli 20 Özellik (Feature Importance)', fontsize=16)
plt.xlabel('Önem Skoru', fontsize=12)
plt.ylabel('Özellikler', fontsize=12)
plt.show()